In [6]:
#1. rutin1 import module
import pandas as pd
import os
import sys

In [7]:
#2. rutin2 membuat syspath ke root utk aktifkan __init__.py
repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
sys.path.append(repo_root)

In [8]:
#3. rutin3 Load the dataset
dataset_path = os.path.join(repo_root, "data/dataset_ekstraksi_r29_lg.xlsx")
df = pd.read_excel(dataset_path)
df["text"] = df["judul"] +". "+ df["isi"]

In [9]:
# tuning paramater
tuning_multiplier = 1  #aktor pengali dari score jika kata tersebut merupakan frase. default = 1 (perlu variasi 0.6 - 0.75)
tuning_f_phrase = 3  #score minimum utk bisa disebut frase
m_prediksi = 3  #jumlah top -n keyword prediksi
n_top_phrase = 3   #jumlah frase yg akan di cari dalam fungsi get_top_phrase

In [10]:
# Preprocess
import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemover, ArrayDictionary

def preprocess(text):
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    text = text.replace('.', '. ')
    text = re.sub('[^a-zA-Z.]', ' ', text)
    text = text.lower()
    text = re.sub("(\\d|\\W)+"," ",text)
    text = text.strip()

    #stopwords_path = os.path.join(repo_root, "data/all_stop_words.txt")
    stopwords_path = os.path.join(repo_root, "notebooks/stopwords_tuning/all_stop_words.txt")
    with open(stopwords_path, 'r') as f:
        stopwords = [line.strip() for line in f]

    dictionary = ArrayDictionary(stopwords)
    str = StopWordRemover(dictionary)
    text = str.remove(text)

    return text

df['preprocessed_text'] = df['text'].apply(preprocess)
df_tr = df['preprocessed_text'].apply(preprocess)

In [11]:
# Predict keywords for all sentences in the dataframe and save the in dataframe
from utils import build_graph, score_phrases, get_top_phrase, predict_keywords, write_excel

predict_textrank = pd.DataFrame()
for i in df_tr.index:
    keyphrase = predict_keywords(df_tr[i], m_prediksi, tuning_f_phrase, tuning_multiplier).reset_index(drop=True)
    a = pd.DataFrame(keyphrase.Keyword).T.reset_index(drop=True)
    b = pd.DataFrame(keyphrase.Score).round(2).T.reset_index(drop=True)
    keyphrase = pd.concat([a, b], axis=1)

    predict_textrank = pd.concat([predict_textrank, keyphrase], ignore_index=True)
predict_textrank.columns = ['key_1', 'key_2','key_3','score_1', 'score_2','score_3'] 

EVALUATION

In [12]:
from utils import eval

targets = df[["k1", "k2", "k3","k4", "k5", "k6","k7"]].values.tolist()
df_targets = pd.DataFrame(targets)

In [13]:
# Evaluation TextRank
predict_textrank_list = predict_textrank[['key_1','key_2','key_3']].values.tolist()
eval_textrank = eval(predict_textrank_list, targets, True).round(3)
eval_textrank.columns = ['key_1', 'key_2','key_3','strict_recall', 'strict_prec', 'flex_recall','flex_prec']
eval_textrank = eval_textrank[['key_1', 'key_2','key_3', 'flex_recall','flex_prec']] # untuk menyederhanakan hasil evaluasi
eval_textrank.head(3)

,key_1,key_2,key_3,flex_recall,flex_prec
0,no_match,no_match,no_match,0.000,0.000
1,partial_match,partial_match,full_match,0.429,1.000
2,no_match,partial_match,no_match,0.143,0.333


In [14]:
# Calculate TextRank Score, using flexible score : exact maatch =1, partial match = 1, no match = 0
textrank_recall = eval_textrank['flex_recall'].mean()
textrank_prec = eval_textrank['flex_prec'].mean()
textrank_f1 = 2 * (textrank_prec * textrank_recall) / (textrank_prec + textrank_recall)

# Create a DataFrame with the scores
summary = pd.DataFrame({'textrank': [textrank_recall, textrank_prec, textrank_f1]}, index=['recall', 'precision', 'F1'])
summary = summary.round(3)
summary

,textrank
recall,0.117
precision,0.273
F1,0.164


In [15]:
# Combine dataframe predict_textrank, df_targets and eval_textrank
predict_textrank = pd.concat([predict_textrank, df_targets, eval_textrank], axis=1)
predict_textrank.head(3)

,key_1,key_2,key_3,score_1,score_2,score_3,0,1,2,3,4,5,6,key_1,key_2,key_3,flex_recall,flex_prec
0,personil diajukan fase,diajukan fase tender,organisasi diajukan fase,9.80,7.46,7.42,persetujuan tertulis,prosedur,usulan,pengganti,NaN,NaN,NaN,no_match,no_match,no_match,0.000,0.000
1,document,template,pengelolaan,1.53,1.26,1.12,template document,exhibit c,acuan,pengelolaan,dokumen,NaN,NaN,partial_match,partial_match,full_match,0.429,1.000
2,ruangan kantor,kantor,ruangan,5.30,2.97,2.33,ruang kantor,change inquiry,lingkup kerja,akomodasi,services for company,exhibit a,NaN,no_match,partial_match,no_match,0.143,0.333


In [16]:
# Write predictions to excel file
sheet_name = 'tr_phrase'
output_file = 'std_ia_textrank.xlsx'
write_excel(predict_textrank, sheet_name, output_file)

/Users/jim/Documents/GitHub/kw_ina_extraction/utils/ia_file_operation.py:16: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book
/Users/jim/Documents/GitHub/kw_ina_extraction/utils/ia_file_operation.py:25: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
